<a href="https://colab.research.google.com/github/sarthakb4/Data_science_lab/blob/main/Exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Graph (Adjacency List):")
for node, neighbors in graph.items():
    print(f"{node}: {neighbors}")

Graph (Adjacency List):
A: ['B', 'C']
B: ['A', 'D', 'E']
C: ['A', 'F']
D: ['B']
E: ['B', 'F']
F: ['C', 'E']


In [2]:
from collections import deque

def bfs(graph, start_node):
    visited = set() # To keep track of visited nodes
    queue = deque([start_node]) # Initialize a queue with the start node
    visited.add(start_node) # Mark the start node as visited

    bfs_traversal = []

    while queue:
        current_node = queue.popleft() # Dequeue a node
        bfs_traversal.append(current_node)

        # Enqueue all unvisited neighbors
        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)

    return bfs_traversal


print("\nBFS Traversal starting from node 'A':")
print(bfs(graph, 'A'))


BFS Traversal starting from node 'A':
['A', 'B', 'C', 'D', 'E', 'F']


In [3]:
def dfs(graph, start_node):
    visited = set() # To keep track of visited nodes
    stack = [start_node] # Initialize a stack with the start node
    dfs_traversal = []

    while stack:
        current_node = stack.pop() # Pop a node from the stack

        if current_node not in visited:
            visited.add(current_node)
            dfs_traversal.append(current_node)

            # Push unvisited neighbors onto the stack (order might vary depending on implementation,
            # reversing ensures consistent output if neighbors are sorted)
            for neighbor in sorted(graph[current_node], reverse=True):
                if neighbor not in visited:
                    stack.append(neighbor)

    return dfs_traversal


print("\nDFS Traversal starting from node 'A':")
print(dfs(graph, 'A'))


DFS Traversal starting from node 'A':
['A', 'B', 'D', 'E', 'F', 'C']


In [4]:
graph_astar = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 1},
    'D': {'B': 2},
    'E': {'B': 5, 'F': 1},
    'F': {'C': 1, 'E': 1}
}

# Heuristic values (estimated cost from node to goal 'F')
heuristics = {
    'A': 6,
    'B': 4,
    'C': 1,
    'D': 6,
    'E': 1,
    'F': 0  # Goal node has 0 heuristic cost
}

print("Graph for A* (Adjacency List with Costs):")
for node, neighbors in graph_astar.items():
    print(f"{node}: {neighbors}")

print("\nHeuristics (to goal 'F'):")
for node, h_val in heuristics.items():
    print(f"{node}: {h_val}")

Graph for A* (Adjacency List with Costs):
A: {'B': 1, 'C': 4}
B: {'A': 1, 'D': 2, 'E': 5}
C: {'A': 4, 'F': 1}
D: {'B': 2}
E: {'B': 5, 'F': 1}
F: {'C': 1, 'E': 1}

Heuristics (to goal 'F'):
A: 6
B: 4
C: 1
D: 6
E: 1
F: 0


In [5]:
import heapq

def astar_search(graph, start, goal, h):
    open_set = [] # Priority queue to store (f_cost, node)
    heapq.heappush(open_set, (0 + h[start], start)) # (f_cost, node)

    came_from = {} # To reconstruct the path

    g_score = {node: float('inf') for node in graph} # Actual cost from start to node
    g_score[start] = 0

    f_score = {node: float('inf') for node in graph} # Estimated total cost from start to goal through node
    f_score[start] = h[start]

    while open_set:
        current_f_cost, current_node = heapq.heappop(open_set)

        if current_node == goal:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1] # Reverse to get path from start to goal

        for neighbor, weight in graph[current_node].items():
            tentative_g_score = g_score[current_node] + weight

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + h[neighbor]
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None # No path found


print("\nA* Path from 'A' to 'F':")
path_astar = astar_search(graph_astar, 'A', 'F', heuristics)
if path_astar:
    print(f"Path: {path_astar}")
else:
    print("No path found.")


A* Path from 'A' to 'F':
Path: ['A', 'C', 'F']


In [6]:
# A simple game tree represented as a dictionary
# 'children': list of child nodes
# 'value': score if it's a terminal node (leaf)

game_tree = {
    'A': {'children': ['B', 'C']},
    'B': {'children': ['D', 'E']},
    'C': {'children': ['F', 'G']},
    'D': {'value': 3},
    'E': {'value': 12},
    'F': {'value': 8},
    'G': {'value': 2}
}

print("Simple Game Tree:")
for node, data in game_tree.items():
    print(f"{node}: {data}")

Simple Game Tree:
A: {'children': ['B', 'C']}
B: {'children': ['D', 'E']}
C: {'children': ['F', 'G']}
D: {'value': 3}
E: {'value': 12}
F: {'value': 8}
G: {'value': 2}


In [7]:
def minimax(node_id, is_maximizing_player, tree):
    node = tree[node_id]

    # If it's a terminal node, return its value
    if 'value' in node:
        return node['value']

    if is_maximizing_player:
        max_eval = -float('inf')
        for child_id in node['children']:
            eval = minimax(child_id, False, tree)  # Opponent (minimizing player) plays next
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for child_id in node['children']:
            eval = minimax(child_id, True, tree)  # Opponent (maximizing player) plays next
            min_eval = min(min_eval, eval)
        return min_eval


print("\nRunning Minimax for the game tree:")
# Assume 'A' is the root and the maximizing player starts
optimal_value = minimax('A', True, game_tree)
print(f"The optimal value for the maximizing player at the root ('A') is: {optimal_value}")


Running Minimax for the game tree:
The optimal value for the maximizing player at the root ('A') is: 3
